# 1次元の畳み込みニューラルネットワークスクラッチ

畳み込みニューラルネットワーク（CNN） のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。


このSprintでは1次元の 畳み込み層 を作成し、畳み込みの基礎を理解することを目指します。次のSprintでは2次元畳み込み層とプーリング層を作成することで、一般的に画像に対して利用されるCNNを完成させます。


クラスの名前はScratch1dCNNClassifierとしてください。クラスの構造などは前のSprintで作成したScratchDeepNeuralNetrowkClassifierを参考にしてください。

## 1次元畳み込み層とは

CNNでは画像に対しての2次元畳み込み層が定番ですが、ここでは理解しやすくするためにまずは1次元畳み込み層を実装します。1次元畳み込みは実用上は自然言語や波形データなどの 系列データ で使われることが多いです。


畳み込みは任意の次元に対して考えることができ、立体データに対しての3次元畳み込みまではフレームワークで一般的に用意されています。

In [2]:
from keras.datasets import mnist
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

## データセットの用意

検証には引き続きMNISTデータセットを使用します。1次元畳み込みでは全結合のニューラルネットワークと同様に平滑化されたものを入力します。

## 【問題1】チャンネル数を1に限定した1次元畳み込み層クラスの作成

チャンネル数を1に限定した1次元畳み込み層のクラスSimpleConv1dを作成してください。基本構造は前のSprintで作成した全結合層のFCクラスと同じになります。なお、重みの初期化に関するクラスは必要に応じて作り変えてください。Xavierの初期値などを使う点は全結合層と同様です。


ここでは パディング は考えず、ストライド も1に固定します。また、複数のデータを同時に処理することも考えなくて良く、バッチサイズは1のみに対応してください。この部分の拡張はアドバンス課題とします。


フォワードプロパゲーションの数式は以下のようになります。

$$a_i = \sum_{s=0}^{F-1}x_{(i+s)}w_s+b$$

$
a
i
$ : 出力される配列のi番目の値

$
F
$ : フィルタのサイズ

$
x
(
i
+
s
)
 $: 入力の配列の(i+s)番目の値

$
w
s
$ : 重みの配列のs番目の値

$
b
$ : バイアス項


全てスカラーです。


次に更新式です。ここがAdaGradなどに置き換えられる点は全結合層と同様です。

$$w_s^{\prime} = w_s - \alpha \frac{\partial L}{\partial w_s} \\
b^{\prime} = b - \alpha \frac{\partial L}{\partial b}$$

$
α
 $: 学習率


$\frac{\partial L}{\partial w_s}$
 : $
w
s
$ に関する損失 $
L
$ の勾配

$\frac{\partial L}{\partial b}$
 : $
b
$ に関する損失 $
L
$ の勾配


勾配 
$\frac{\partial L}{\partial w_s}$
 や 
$\frac{\partial L}{\partial b}$

 を求めるためのバックプロパゲーションの数式が以下です
 
 
$$\frac{\partial L}{\partial w_s} = \sum_{i=0}^{N_{out}-1} \frac{\partial L}{\partial a_i}x_{(i+s)}\\
\frac{\partial L}{\partial b} = \sum_{i=0}^{N_{out}-1} \frac{\partial L}{\partial a_i}$$ 

$\frac{\partial L}{\partial a_i}$
: 勾配の配列のi番目の値

$N_{out}$
: 出力のサイズ

前の層に流す誤差の数式は以下です。

$$\frac{\partial L}{\partial x_j} = \sum_{s=0}^{F-1} \frac{\partial L}{\partial a_{(j-s)}}w_s$$

$\frac{\partial L}{\partial x_j}$
: 前の層に流す誤差の配列のj番目の値

ただし、$j-s<0$ または  $j-s>N_{out}-1$  のとき $\frac{\partial L}{\partial a_{(j-s)}} =0$　です。


全結合層との大きな違いは、重みが複数の特徴量に対して共有されていることです。この場合は共有されている分の誤差を全て足すことで勾配を求めます。計算グラフ上での分岐はバックプロパゲーションの際に誤差の足し算をすれば良いことになります。
 

In [ ]:
# テスト
x = np.array([1, 2, 3, 4])
w = np.array([3, 5, 7])
b = np.array([1])

In [ ]:
#配列作成
idx1 = np.arange(w.shape[0])
idx2 = np.arange(w.shape[0] - 1).reshape(-1, 1)
index = idx1 + idx2
a = np.dot(x[index], w.T) + b
print(a)

# a = np.array([35, 50])

In [ ]:
da = np.array([10, 20])
d = np.sum(da)
d

#delta_b = np.array([30])

In [ ]:
s = np.dot(da,x[index])
print(da)
print(x[index])
s

#d elta_w = np.array([50, 80, 110])

In [ ]:
#0配列を用意
dx = np.zeros(x.shape[0])


for j in range(x.shape[0]):    
    for s in range(w.shape[0]):   
        
        # j 0,0,0,1,1,1....
        # s 0,1,2,0,1,2....
        # 0未満　or １超える
        if j - s < 0 or j - s > 1:
            
            # w[s] = 3,5,7,3,5,7....
            # w[s] = 条件該当
            dx[j] += 0 * w[s]

        # その他(0,1)
        else:
            
            # da[j-s]は該当値*3,5,7
            dx[j] += da[j - s] * w[s]

dx

# delta_x = np.array([30, 110, 170, 140])

In [3]:
class SimpleConv1d:
    def __init__(self, W, b, stride=1, pad=0):
        self.W, self.b = W, b
        self.stride = stride
        self.pad = pad
        self.X =None
        self.col = None
        self.col_W = None
        self.dW = None
        self.db = None

    def forward(self, X):

        FN, C, FL = self.W.shape #n_filters, n_channels, filter_length
        N, C, L = X.shape #batch_size, n_channels, n_features
        out_size = get_output_size(L, FL, stride=self.stride, pad=self.pad)
        col = im2col_1d(input_data=X, filter_size=FL, stride=self.stride, pad=self.pad)

        out = col @ self.W.reshape(FN, -1).T + self.b
        out = out.reshape(N, out_size, -1).transpose(0, 2, 1)
        self.X = X
        self.col = col
        return out


    def backward(self, dout):
        FN, C, FL = self.W.shape #n_filters, n_channels, filter_length
        N, C, L = self.X.shape
        dout = dout.transpose(0, 2, 1).reshape(-1, FN)

        self.db = np.sum(dout, axis=0)
        self.dW = self.col.T @ dout
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FL)
        dcol = dout @ self.W.reshape(FN, -1)

        dx = col2im_1d(col=dcol, input_shape=self.X.shape, filter_size=FL, stride=self.stride, pad=self.pad)
        return dx

## 【問題2】1次元畳み込み後の出力サイズの計算

畳み込みを行うと特徴量の数が変化します。どのように変化するかは以下の数式から求められます。パディングやストライドも含めています。この計算を行う関数を作成してください。

$$ N_{out} =  \frac{N_{in}+2P-F}{S} + 1\\ $$

$N_{out}$
 : 出力のサイズ（特徴量の数）


$N_{in}$
 : 入力のサイズ（特徴量の数）

$
P
$ : ある方向へのパディングの数

$
F
$ : フィルタのサイズ


$S$
 : ストライドのサイズ

In [4]:
def get_output_size(n_features, filter_length, stride=1, pad=0):
    return int(1 + (n_features + 2 * pad - filter_length) / stride)

In [5]:
def im2col_1d(input_data, filter_size, stride=1, pad=0):
    """
    Paramaters
    ----------------
    input_data: ndarray of shape(n_data, channel, n_features)
    filter_size: int
    stride: int
    pad: int
    Returns
    -----------------
    col: 1D array
    """

    N, C, L = input_data.shape#
    out_size = get_output_size(n_features=L, filter_length=filter_size, stride=stride, pad=pad)

    #padding
    img = np.pad(input_data, [(0, 0), (0, 0), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_size, out_size))

    for f in range(filter_size):
        f_max = f + stride * out_size
        col[:, :, f, :] = img[:, :, f:f_max:stride]

    col = col.transpose(0, 3, 1, 2)
    col = col.reshape(N * out_size, -1)
    return col

def col2im_1d(col, input_shape, filter_size, stride=1, pad=0):
    """
    Parameters
    -----------------
    col: data to convert
    input_shape:
        i.e. shape(10, 1, 784)
    filter_size: int
    stride: int
    pad: int
    Returns
    -----------------
    ndarray of shape input_shape
    """
    N, C, L = input_shape#shape(n_data, channel, n_features)
    out_size = get_output_size(n_features=L, filter_length=filter_size, stride=stride, pad=pad)

    col = col.reshape(N, out_size, C, filter_size).transpose(0, 2, 3, 1)
    img = np.zeros((N, C, L + 2 * pad +  stride - 1))
    for f in range(filter_size):
        f_max = f + stride * out_size
        img[:, :, f:f_max:stride] += col[:, :, f, :]

    return img[:, :, pad:L + pad]

## 【問題3】小さな配列での1次元畳み込み層の実験

次に示す小さな配列でフォワードプロパゲーションとバックプロパゲーションが正しく行えているか確認してください。


入力x、重みw、バイアスbを次のようにします。

    x = np.array([1,2,3,4])
    w = np.array([3, 5, 7])
    b = np.array([1])
    
フォワードプロパゲーションをすると出力は次のようになります

    a = np.array([35, 50])
    
    
次にバックプロパゲーションを考えます。誤差は次のようであったとします。

    delta_a = np.array([10, 20])
    
バックプロパゲーションをすると次のような値になります。

    delta_b = np.array([30])
    delta_w = np.array([50, 80, 110])
    delta_x = np.array([30, 110, 170, 140])
    


In [ ]:
X = np.array([[[1, 2, 3, 4]]])
W = np.array([[[3, 5, 7]]])
b = np.array([1])
delta_a = np.array([[[10, 20]]])

s = SimpleConv1d(W=W,b=b)
sc = s.forward(X)
print(sc)

dx = s.backward(delta_a)
print(s.db)
print(s.dW)
print(dx)

## 実装上の工夫

畳み込みを実装する場合は、まずはfor文を重ねていく形で構いません。しかし、できるだけ計算は効率化させたいため、以下の式を一度に計算する方法を考えることにします。

$$ a_i = \sum_{s=0}^{F-1}x_{(i+s)}w_s+b $$

バイアス項は単純な足し算のため、重みの部分を見ます。

$$ \sum_{s=0}^{F-1}x_{(i+s)}w_s $$

これは、xの一部を取り出した配列とwの配列の内積です。具体的な状況を考えると、以下のようなコードで計算できます。この例では流れを分かりやすくするために、各要素同士でアダマール積を計算してから合計を計算しています。これは結果的に内積と同様です。

    x = np.array([1, 2, 3, 4])
    w = np.array([3, 5, 7])
    a = np.empty((2, 3))
    indexes0 = np.array([0, 1, 2]).astype(np.int)
    indexes1 = np.array([1, 2, 3]).astype(np.int)
    a[0] = x[indexes0]*w # x[indexes0]は([1, 2, 3])である
    a[1] = x[indexes1]*w # x[indexes1]は([2, 3, 4])である
    a = a.sum(axis=1)
    
ndarrayは配列を使ったインデックス指定ができることを利用した方法です。


また、二次元配列を使えば一次元配列から二次元配列が取り出せます。

    x = np.array([1, 2, 3, 4])
    indexes = np.array([[0, 1, 2], [1, 2, 3]]).astype(np.int)
    print(x[indexes]) # ([[1, 2, 3], [2, 3, 4]])


このこととブロードキャストなどをうまく組み合わせることで、一度にまとめて計算することも可能です。


畳み込みの計算方法に正解はないので、自分なりに効率化していってください。


《参考》


以下のページのInteger array indexingの部分がこの方法についての記述です。


[Indexing — NumPy v1.17 Manual](https://numpy.org/doc/stable/reference/arrays.indexing.html)



## 【問題4】チャンネル数を限定しない1次元畳み込み層クラスの作成

チャンネル数を1に限定しない1次元畳み込み層のクラスConv1dを作成してください。


例えば以下のようなx, w, bがあった場合は、

    x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。
    w = np.ones((3, 2, 3)) # 例の簡略化のため全て1とする。(出力チャンネル数、入力チャンネル数、フィルタサイズ)である。
    b = np.array([1, 2, 3]) # （出力チャンネル数）

出力は次のようになります。

    a = np.array([[16, 22], [17, 23], [18, 24]]) # shape(3, 2)で、（出力チャンネル数、特徴量数）である。

入力が2チャンネル、出力が3チャンネルの例です。計算グラフを書いた上で、バックプロパゲーションも手計算で考えてみましょう。計算グラフの中には和と積しか登場しないので、微分を新たに考える必要はありません。


《補足》


チャンネル数を加える場合、配列をどういう順番にするかという問題があります。(バッチサイズ、チャンネル数、特徴量数)または(バッチサイズ、特徴量数、チャンネル数)が一般的で、ライブラリによって順番は異なっています。（切り替えて使用できるものもあります）


今回のスクラッチでは自身の実装上どちらが効率的かを考えて選んでください。上記の例ではバッチサイズは考えておらず、(チャンネル数、特徴量数)です。

In [ ]:
X = np.array([[[1, 2, 3, 4], [2, 3, 4, 5]]])
W = np.ones((3, 2, 3))
b = np.array([1, 2, 3])
a = np.array([[16, 22], [17, 23], [18, 24]]) 


fc = SimpleConv1d(W=W,b=b)
te = fc.forward(X)
print(te)

bk = fc.backward(te)
print(bk)

## 【問題5】（アドバンス課題）パディングの実装

畳み込み層にパディングの機能を加えてください。1次元配列の場合、前後にn個特徴量を増やせるようにしてください。


最も単純なパディングは全て0で埋める ゼロパディング であり、CNNでは一般的です。他に端の値を繰り返す方法などもあります。


フレームワークによっては、元の入力のサイズを保つようにという指定をすることができます。この機能も持たせておくと便利です。なお、NumPyにはパディングの関数が存在します。


[numpy.pad — NumPy v1.17 Manual](https://numpy.org/doc/stable/reference/generated/numpy.pad.html)

In [ ]:
pd = SimpleConv1d(W=W,b=b,stride=1,pad=1)
te1 = pd.forward(X)
print(te1)
te2 = pd.backward(te1)
print(te2)

## 【問題6】（アドバンス課題）ミニバッチへの対応

ここまでの課題はバッチサイズ1で良いとしてきました。しかし、実際は全結合層同様にミニバッチ学習が行われます。Conv1dクラスを複数のデータが同時に計算できるように変更してください。



問題１で実装

## 【問題7】（アドバンス課題）任意のストライド数

ストライドは1限定の実装をしてきましたが、任意のストライド数に対応できるようにしてください。



In [ ]:
pd = SimpleConv1d(W=W,b=b,stride=3,pad=0)
te3 = pd.forward(X)
print(te3)
te4 = pd.backward(te3)
print(te4)

## 【問題8】学習と推定

これまで使ってきたニューラルネットワークの全結合層の一部をConv1dに置き換えてMNISTを学習・推定し、Accuracyを計算してください。


出力層だけは全結合層をそのまま使ってください。ただし、チャンネルが複数ある状態では全結合層への入力は行えません。その段階でのチャンネルは1になるようにするか、 平滑化 を行なってください。


画像に対しての1次元畳み込みは実用上は行わないことのため、精度は問いません。

In [6]:
class GetMiniBatch:
        """
        ミニバッチを取得するイテレータ
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
          学習データ
        y : 次の形のndarray, shape (n_samples, 1)
          正解値
        batch_size : int
          バッチサイズ
        seed : int
          NumPyの乱数のシード
        """
        def __init__(self, X, y, batch_size=10, seed=0):
                self.batch_size = batch_size
                np.random.seed(seed)
                shuffle_index = np.random.permutation(np.arange(X.shape[0]))
                self.X = X[shuffle_index]
                self.y = y[shuffle_index]
                self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)

        def __len__(self):
                return self._stop

        def __getitem__(self,item):
                p0 = item*self.batch_size
                p1 = item*self.batch_size + self.batch_size
                return self.X[p0:p1], self.y[p0:p1]        

        def __iter__(self):
                self._counter = 0
                return self

        def __next__(self):
                if self._counter >= self._stop:
                        raise StopIteration()
                p0 = self._counter*self.batch_size
                p1 = self._counter*self.batch_size + self.batch_size
                self._counter += 1
                return self.X[p0:p1], self.y[p0:p1]

In [7]:
class ReLU():
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = np.copy(x)
        out[self.mask] = 0
        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

class Softmax():
    def __init__(self):
        pass

    def forward(self, x):
        x = x - np.max(x, axis=1, keepdims=True)
        #softmax
        out = np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True)
        return out

    def backward(self, dout):

        batch_size = dout.shape[0]
        dx = (dout) / batch_size
        return dx

In [8]:
class SGD():
    #stochastic gradient descent
    def __init__(self, lr=0.01):
        self.lr = lr
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]
            
class AdaGrad():
    #AdaGrad
    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None
    def update(self, params, grads):
        if (self.h is None):
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
        for key in params.keys():
            self.h[key] += grads[key] * grads[key]
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)
            

class Adam():
    #Adam
    def __init__(self, lr=0.01, beta1=0.9, beta2=0.999):
        self.lr =lr
        self.beta1=beta1
        self.beta2=beta2
        self.iter = 0
        self.m = None
        self.v = None
    def update(self, params, grads):
        if (self.m is None):
            self.m, self.v = {}, {}
            for key, val in params.items():
                self.m[key] = np.zeros_like(val)
                self.v[key] = np.zeros_like(val)
        self.iter += 1
        lr_t = self.lr * np.sqrt(1.0 - self.beta2 ** self.iter) / (1.0 - self.beta1 **self.iter)

        for key in params.keys():
            self.m[key] += (1 - self.beta1) * (grads[key] - self.m[key])
            self.v[key] += (1 - self.beta2) * (grads[key] ** 2 - self.v[key])
            params[key] -= lr_t * self.m[key] / (np.sqrt(self.v[key]) + 1e-7)

In [9]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x):

        self.x = x
        out = self.x @ self.W + self.b
        return out

    def backward(self, dout):
        dx = dout @ self.W.T
        self.dW = self.x.T @ dout
        self.db = np.sum(dout, axis=0)
        return dx

class Flatten():
    def __init__(self):
        self.prev_layer_shape = None

    def forward(self, x):
        self.prev_layer_shape = x.shape
        #(N, C, H, W)を(N, C*H*W)に
        return x.reshape((x.shape[0], -1))

    def backward(self, dout):
        #(N, C*H*W)を(N, C, H, W)に
        return (dout.reshape(self.prev_layer_shape))
    
class MaxPooling1D():
    def __init__(self, pool_size, stride=1, pad=0):
        self.pool_size = pool_size
        self.stride = stride
        self.pad = pad
        self.x = None
        self.arg_max=None

    def forward(self, x):
        N, C, L = x.shape #batch_size, n_channels, n_features
        out_size = get_output_size(L, self.pool_size, stride=self.stride, pad=self.pad)

        col = im2col_1d(x, self.pool_size, self.stride, self.pad)
        col = col.reshape(-1, self.pool_size)

        arg_max = np.argmax(col, axis=1)
        out = np.max(col, axis=1)
        out = out.reshape(N, out_size, C).transpose(0, 2, 1)

        self.x = x
        self.arg_max = arg_max
        return out

    def backward(self, dout):
        dout = dout.transpose(0, 2, 1)
        dmax = np.zeros((dout.size, self.pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (self.pool_size, ))
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[2], -1)
        dx = col2im_1d(col=dcol, input_shape=self.x.shape, filter_size=self.pool_size, stride=self.stride, pad=self.pad)
        return dx

In [13]:
from collections import OrderedDict

class Scratch1dCNNClassifier():
    def __init__(
        self, conv_param={'n_filters': 30, 'filter_size': 3, 'stride': 1, 'pad': 0},
        pool_param={'pool_size': 2},
        n_epochs=5, batch_size=100, optimizer='Adam',
        optimizer_param={'lr': 0.001},
        layer_nodes = {'hidden': 100, 'output': 10},
        weight_init_std=0.01,
        verbose=True
    ):
        self.conv_param = conv_param
        self.pool_param = pool_param
        self.layer_nodes = layer_nodes
        self.weight_init_std = weight_init_std
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self.verbose = verbose

        optimizer_class_dict = {'sgd': SGD, 'adagrad': AdaGrad, 'adam': Adam}
        #**kwargsでdictで引数をまとめて受け取っている
        self.optimizer = optimizer_class_dict[optimizer.lower()](**optimizer_param)

        self.train_loss_list =[]
        self.train_acc_list = []
        self.val_loss_list = []
        self.val_acc_list = []

    def fit(self, x_train, y_train, x_val=None, y_val=None):
        self.x_train = x_train
        self.y_train = y_train
        self.x_val = x_val
        self.y_val = y_val
        #layerを生成
        self._gen_layers()
        #epoch数だけ学習
        for epoch in range(self.n_epochs):
            self._train()
            print("epoch: " + str(epoch))
            #verbose=Trueなら学習中のlossなど計算して表示
            if (self.verbose):
                self._calc_loss_acc()
                print("train_acc: " + str(self.train_acc_list[epoch]) + ", val_acc" + str(self.val_acc_list[epoch]))
                print("train loss: " + str(self.train_loss_list[epoch]) + ", val_loss" + str(self.val_loss_list[epoch]) )
        return self.train_loss_list, self.train_loss_list

    def predict(self, x):
        proba = self._propagate_forward(x)
        return np.argmax(proba, axis=1)

    def _gen_layers(self):
        """
        x_train: ndarray of shape(n_samples, n_channels, n_features)
        """
        self.n_train_samples, n_channels, n_features = self.x_train.shape
        n_filters = self.conv_param['n_filters']
        filter_size = self.conv_param['filter_size']
        filter_stride = self.conv_param['stride']
        filter_pad = self.conv_param['pad']
        pool_size = self.pool_param['pool_size']

        conv_output_size = get_output_size(n_features, filter_size, filter_stride, filter_pad)
        pool_output_size = int(n_filters * conv_output_size/ pool_size)

        #initialize hyper parameters
        self.params ={}
        self.params['W1'] = self.weight_init_std * np.random.randn(n_filters, n_channels, filter_size)
        self.params['b1'] = np.zeros(n_filters)
        self.params['W2'] = self.weight_init_std * np.random.randn(pool_output_size, self.layer_nodes['hidden'])
        self.params['b2'] = np.zeros(self.layer_nodes['hidden'])
        self.params['W3'] = self.weight_init_std * np.random.randn(self.layer_nodes['hidden'], self.layer_nodes['output'])
        self.params['b3'] =  np.zeros(self.layer_nodes['output'])

        #generate layers
        self.layers = OrderedDict()
        self.layers['Conv1'] = SimpleConv1d(self.params['W1'], self.params['b1'], filter_stride, filter_pad)
        self.layers['Relu1'] = ReLU()
        self.layers['Pool1'] = MaxPooling1D(pool_size=pool_size, stride=pool_size)
        self.layers['Flatten1'] = Flatten()
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = ReLU()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        self.layers['Last'] = Softmax()

        #gradients
        self.grads = {}

    def _train(self):
        mini_batch = GetMiniBatch(X=self.x_train, y=self.y_train, batch_size=self.batch_size, seed=0)

        for mini_x, mini_y in mini_batch:
            #forward
            z = self._propagate_forward(mini_x)
            #backward
            self._propagate_backward(z - mini_y)
            #gradient更新
            self.optimizer.update(self.params, self.grads)
        return

    def _loss(self, y_actual, pred_proba):
        return -(y_actual * np.log(pred_proba + 1e-7)).sum() / y_actual.shape[0]

    def _accuracy(self, y_actual, pred_proba):
        y_actual = np.argmax(y_actual, axis=1)
        pred = np.argmax(pred_proba, axis=1)
        acc = np.sum( y_actual == pred) / y_actual.shape[0]
        return acc

    def _propagate_forward(self, x):
        #forward
        for layer in self.layers.values():
            x = layer.forward(x)
        return x

    def _propagate_backward(self, dout):
        #backward
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        self.grads['W1'], self.grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        self.grads['W2'], self.grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        self.grads['W3'], self.grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
        #返却値なし
        return
    def _calc_loss_acc(self):
        proba = self._propagate_forward(self.x_train)
        #loss計算
        loss = self._loss(self.y_train, proba)
        self.train_loss_list.append(loss)
        #accuracy計算
        train_acc = self._accuracy(self.y_train, proba)
        self.train_acc_list.append(train_acc)

        if((self.x_val is not None) & (self.y_val is not None)):
            proba = self._propagate_forward(self.x_val)
            #loss計算
            val_loss = self._loss(self.y_val, proba)
            self.val_loss_list.append(loss)
            #accuracy計算
            val_acc = self._accuracy(self.y_val, proba)
            self.val_acc_list.append(val_acc)
        #返却値なし
        return

In [14]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#data数を絞る
X_train = X_train[:30000]
y_train = y_train[:30000]
X_test = X_test[:500]
y_test = y_test[:500]

#reshape(N, 28, 28) -> (N, 1, 784)
X_train = X_train.reshape(-1, 1, X_train.shape[1]*X_train.shape[2])
X_test = X_test.reshape(-1, 1, X_test.shape[1]*X_test.shape[2])

#normalize
X_train = X_train.astype(np.float)/255
X_test = X_test.astype(np.float)/255
X_train.shape
#one-hot
eye = np.eye(len(np.unique(y_train)))
y_train = eye[y_train]
y_test = eye[y_test]

#split into train, val
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
print(X_train.shape) 
print(X_val.shape)

(24000, 1, 784)
(6000, 1, 784)


In [15]:
cnn = Scratch1dCNNClassifier(
    conv_param={'n_filters': 30, 'filter_size': 5, 'stride': 1, 'pad': 0},
    pool_param={'pool_size': 2},
    n_epochs=3,
    batch_size=1000,
    ba='Adam',
    optimizer_param={'lr': 0.001},
    layer_nodes = {'hidden': 100, 'output': 10},
    weight_init_std=0.01,
    verbose=True
)

cnn.fit(x_train=X_train, y_train=y_train, x_val=X_val, y_val=y_val)
pred = cnn.predict(X_test)

epoch: 0
train_acc: 0.7156666666666667, val_acc0.7238333333333333
train loss: 1.3920215719357045, val_loss1.3920215719357045
epoch: 1
train_acc: 0.8407083333333333, val_acc0.8406666666666667
train loss: 0.5195759020137002, val_loss0.5195759020137002
epoch: 2
train_acc: 0.893875, val_acc0.8886666666666667
train loss: 0.3769295664763949, val_loss0.3769295664763949


In [16]:
y_actual = np.argmax(y_test, axis=1)
print("acuracy: ", (pred == y_actual).sum() / len(pred))

acuracy:  0.878


バッチサイズ1000、optimizerをAdamに変更することで改善